In [1]:
#from dotenv import load_dotenv
import os
import time

from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from src.util import build_driver, wait
from src.login import login



In [2]:
# Initialize Chrome Driver
driver = build_driver()

# Load LinkedIn Username and Password
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

# link to the "People" tab of company profile page 
COMPANY_PROFILE_URL = "https://www.linkedin.com/company/endava/people/"

In [3]:
driver.get("https://www.linkedin.com")
time.sleep(3)

login(driver=driver, username=USERNAME, password=PASSWORD)



In [4]:
def get_employee_urls(driver):
    profile_urls = []
    li_elements = driver.find_elements(By.XPATH,"//ul[@class='display-flex list-style-none flex-wrap']/li")

    # Print the text of each li element
    for li in li_elements:
        try:
            a_element = li.find_element(By.XPATH,".//a[@class='app-aware-link ']")
            href_value = a_element.get_attribute("href")
            if '?' in href_value:
                employee_url = href_value.split('?')[0]
            else:
                employee_url = href_value
            print(employee_url)
            profile_urls.append(employee_url)
        except:
            pass
    return profile_urls

def lazy_load(driver):
    # Get the initial height of the page
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait for the page to load
        time.sleep(2)
        
        # Calculate new height and compare with the last height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # Break the loop if the height doesn't change after scrolling
        if new_height == last_height:
            try:
                more_results_btn = driver.find_element(By.XPATH,"//button[@class='artdeco-button artdeco-button--muted artdeco-button--1 artdeco-button--full artdeco-button--secondary ember-view scaffold-finite-scroll__load-button']")
                more_results_btn.click()
                time.sleep(2)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_height = driver.execute_script("return document.body.scrollHeight")
            except NoSuchElementException:
                break
        
        last_height = new_height



In [5]:

def remove_duplicate_lines(string):
    lines = string.splitlines()
    lines = list(set(lines))
    out_string = "\n".join([line.strip() for line in lines])
    out_string = out_string.strip()
    return out_string

def filter_duplicates(experiences):
    out = []
    for experience in experiences:
        duplicate = False
        for element in out:
            if experience in element:
                duplicate = True
                break
        if duplicate:
            continue

        if experience not in out:
            out.append(experience)
    return out


def crawl_employee(driver, profile_url):
    # driver.get(profile_url)
    # time.sleep(3)
    # name_div = driver.find_element(By.XPATH,".//h1[@class='text-heading-xlarge inline t-24 v-align-middle break-words']")
    # name = name_div.text
    # print(name)

    # about_div = driver.find_element(By.XPATH, "//*[@id='profile-content']/div/div[2]/div/div/main/section[4]/div[3]/div/div/div/span[2]")
    # about = about_div.text
    # print(about)

    driver.get(f"{profile_url}/details/experience")
    time.sleep(2)

    experience_container = driver.find_element(By.XPATH, "//div[@class='scaffold-finite-scroll__content']")
    experience_list = experience_container.find_elements(By.XPATH, ".//span[@class='visually-hidden']")
    experiences = ''
    for experience in experience_list:
        experiences = f"{experiences}\n{experience.text}"
    experiences = experiences.strip()
    return experiences

try:
    employees = []
    driver.get(COMPANY_PROFILE_URL)
    lazy_load(driver=driver)
    profile_urls = get_employee_urls(driver=driver)
    for i,_ in enumerate(profile_urls):
        experience = crawl_employee(driver=driver, profile_url=profile_urls[i])
        file_path = f"./data/experience/{i}.txt"
        with open(file_path, "w") as file:
            file.write(experience)
        title = driver.title
        name = title.split('|')[0].strip()
        data = {
            "id": str(i),
            "profile": profile_urls[i],
            "description": file_path,
            "name": name
        }
        employees.append(data)
    
except Exception as e:
    print(e)






https://www.linkedin.com/in/roshinanandra
https://www.linkedin.com/in/nikkiniemczycki
https://www.linkedin.com/in/claire-mannion-024339165
https://www.linkedin.com/in/1-toralpatel
https://www.linkedin.com/in/julia-leavitt
https://www.linkedin.com/in/lynnvan
https://www.linkedin.com/in/alan-odor-54b5b3133
https://www.linkedin.com/in/mikekrolnik
https://www.linkedin.com/in/derick-mullis-72a7b168
https://www.linkedin.com/in/alexisrgold
https://www.linkedin.com/in/boraeristurk
https://www.linkedin.com/in/tatiana-mihaescu
https://www.linkedin.com/in/staceyannhoward
https://www.linkedin.com/in/sarah-w-456a1348
https://www.linkedin.com/in/vicki-littauer-9a141657
https://www.linkedin.com/in/deval-parekh-44172751
https://www.linkedin.com/in/charles-dove-b4715723
https://www.linkedin.com/in/scott-trometer
https://www.linkedin.com/in/dimitry-briskin-pmp-pmi-acp-pal-i-csm-26417530
https://www.linkedin.com/in/adrianroman
https://www.linkedin.com/in/gareth-patterson-6694441ab
https://www.linkedin.co

In [6]:
try:
    import json
    print(len(employees))
    print(employees[0])
    data = {
        "employees": employees
    }
    with open("data/data_2.json", "w") as json_file:
        # Write the dictionary to the JSON file
        json.dump(data, json_file, indent=4)
except:
    pass

982
{'id': '0', 'profile': 'https://www.linkedin.com/in/roshinanandra', 'description': './data/experience/0.txt', 'name': 'Experience'}
